# py_装饰器

版本：R0  
日期：2018-4-25

## 1 装饰器基础

## 2 函数装饰器

## 3 类装饰器
管理类的一种方式。

In [103]:
class Deco:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print("call %s "%(self.func.__name__))
        return self.func(*args, **kwargs)

@Deco
def foo(x, y):
    return x * y

foo(2,3)

call foo 


6

## 4 应用

###  4.1 函数调用次数追踪

In [9]:
# 函数调用次数追踪装饰器
def tracer(func):
    calls = 0
    def wrapper(*args, **kwargs):
        nonlocal calls   # 全局作用域，跟踪函数调用次数
        calls += 1
        print(f"函数{func.__name__}, 调用第 <{calls}> 次")
        return func(*args, **kwargs)
    return wrapper

@tracer
def spam(a,b):
    print(a+b)
    
@tracer
def eggs(x,y,z):
    print(x**y)

In [12]:
spam(1,2)
spam(7,8)

eggs(3,4,5)
eggs(3,4,5)

函数spam, 调用第 <1> 次
3
函数spam, 调用第 <2> 次
15
函数eggs, 调用第 <1> 次
81
函数eggs, 调用第 <2> 次
81


In [101]:
# 类装饰器
class Deco1:
    def __init__(self, func):
        self.count = 0
        self.func = func

    def __call__(self, *args, **kwargs):
        self.count += 1
        print("call %d to %s"%(self.count, self.func.__name__))
        return self.func(*args, **kwargs)

@Deco1
def spam(a, b, c):
    print(a +b +c)
    
@Deco1
def eggs(a, b):
    print(a +b)

In [102]:
spam(1, 2, 3)
spam(1, 2, 3)

eggs(5, 6)
eggs(5, 6)

call 1 to spam
6
call 2 to spam
6
call 1 to eggs
11
call 2 to eggs
11


### 使用描述符装饰方法

In [2]:
class Wrapper:
    def __init__(self, desc, subj):
        self.desc = desc  # Route calls back to decr
        self.subj = subj
        
    def __call__(self, *args, **kwargs):
        return self.desc(self.subj, *args, **kwargs) 

class Tracer(object):
    def __init__(self, func): 
        self.calls = 0  
        self.func = func
        
    def __call__(self, *args, **kwargs): 
        self.calls += 1
        print('call %s to %s' % (self.calls, self.func.__name__))
        return self.func(*args, **kwargs)
    
    def __get__(self, instance, owner): 
        return Wrapper(self, instance)

In [3]:
@Tracer
def spam(a, b, c): 
    print(a +b +c)
    
class Person:
    def __init__(self, name, pay):
        self.name = name
        self.pay = pay
        
    @Tracer
    def giveRaise(self, percent):   # giveRaise = tracer(giverRaise)
        self.pay *= (1.0 + percent)

### 4.3 属性检查装饰器

### 4.4 计时器装饰器

### 4.5 权限管理装饰器

###  变量作用域

In [12]:
def f1(a):
    print(a)
    print(b)

# 没有定义b时，会报错
try:
    f1(1)
except Exception as e:
    print(e)
    
    
# 定义全局b
print(f"{'='*50}")
b = 6
f1(2)


# 作用域
print(f"{'='*50}")
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
try:
    f2(3)
except Exception as e:
    print(e)
    
    
# global 声明
print(f"{'='*50}")
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
f3(3)

1
9
2
6
3
local variable 'b' referenced before assignment
3
6


###  `dis` 模块比较字节码

> `dis`模块为反汇编 python 字节码提供了简单的方式。

In [13]:
from dis import dis

print(dis(f1))
print(f"{'='*50}")
print(dis(f2))
print(f"{'='*50}")
print(dis(f3))

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE
None
 22           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

 23           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

 24          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE
None
 36           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAS

###  闭包

In [20]:
# 计算移动平均值的类

class Averager():
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
    
avg = Averager()
print(f"avg(10)\t{avg(10)}, series\t{avg.series}")
print(f"avg(11)\t{avg(11)}, series\t{avg.series}")
print(f"avg(12)\t{avg(12)}, series\t{avg.series}")

avg(10)	10.0, series	[10]
avg(11)	10.5, series	[10, 11]
avg(12)	11.0, series	[10, 11, 12]


In [37]:
# 计算平均值，保存历史值
def make_averager():
    series = []  # 列表是可变对象，可以读取、更新。对于数字、字符串、元组等不可变类型，只能读取，不可更新，没法实现闭包功能。
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

avg = make_averager()
print(f"avg(10)\t{avg(10)}")
print(f"avg(11)\t{avg(11)}")
print(f"avg(12)\t{avg(12)}")

print(f"{'='*50}")
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)  # 自由变量
print(avg.__code__.co_freevars) 
print(f"{'='*50}")
print(avg.__closure__)   # 各个元素对应于 avg.__closure__[0].cell_contents 中的一个名称， 这些元素是 cell 对象，有cell_contents属性。
print(avg.__closure__[0].cell_contents)

avg(10)	10.0
avg(11)	10.5
avg(12)	11.0
('new_value', 'total')
('series',)
('series',)
(<cell at 0x0000010D214EA468: list object at 0x0000010D21504748>,)
[10, 11, 12]


In [38]:
# 自由变量为数字的情况
# 计算移动平均值,不保存历史值(闭包函数)
def make_averager():
    count = 0  # 列表是可变对象，可以读取、更新。对于数字、字符串、元组等不可变类型，只能读取，不可更新，没法实现闭包功能。
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    return averager

avg = make_averager()
print(f"avg(10)\t{avg(10)}")
print(f"avg(11)\t{avg(11)}")
print(f"avg(12)\t{avg(12)}")

avg(10)	10.0
avg(11)	10.5
avg(12)	11.0


### 计时器装饰器
1. 支持位置、关键字参数；
2. 不遮盖被装饰函数的 `__name__` 和 `__doc__`

In [49]:
import time
import functools

def time_count(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        interval = time.time() - t0
        name = func.__name__
        arg_list = []
        if args:
            arg_list.append(','.join(repr(arg) for arg in args))
        if kwargs:
            kws = [f"{k}={w}" for k, w in sorted(kwargs.items())]
            arg_list.append(', '.join(kws))
        arg_str = ", ".join(arg_list)
        print(f"<{interval:.10f}> {name}({arg_str}) -> {result}")
        return result
    return clocked

In [56]:
@time_count
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

fibonacci(10)

<0.0000000000> fibonacci(0) -> 0
<0.0000000000> fibonacci(1) -> 1
<0.0000000000> fibonacci(2) -> 1
<0.0000000000> fibonacci(1) -> 1
<0.0000000000> fibonacci(0) -> 0
<0.0000000000> fibonacci(1) -> 1
<0.0000000000> fibonacci(2) -> 1
<0.0000000000> fibonacci(3) -> 2
<0.0000000000> fibonacci(4) -> 3
<0.0000000000> fibonacci(1) -> 1
<0.0000000000> fibonacci(0) -> 0
<0.0000000000> fibonacci(1) -> 1
<0.0000000000> fibonacci(2) -> 1
<0.0000000000> fibonacci(3) -> 2
<0.0000000000> fibonacci(0) -> 0
<0.0000000000> fibonacci(1) -> 1
<0.0000000000> fibonacci(2) -> 1
<0.0000000000> fibonacci(1) -> 1
<0.0000000000> fibonacci(0) -> 0
<0.0000000000> fibonacci(1) -> 1
<0.0000000000> fibonacci(2) -> 1
<0.0000000000> fibonacci(3) -> 2
<0.0000000000> fibonacci(4) -> 3
<0.0000000000> fibonacci(5) -> 5
<0.0000000000> fibonacci(6) -> 8
<0.0000000000> fibonacci(1) -> 1
<0.0000000000> fibonacci(0) -> 0
<0.0000000000> fibonacci(1) -> 1
<0.0000000000> fibonacci(2) -> 1
<0.0000000000> fibonacci(3) -> 2
<0.0000000

55

> `functools.lru_cache(maxsize=128, typed=False)`
- `maxsize=128`: 存储多少个调用的结果,建议值为2的幂次方
- `typed=False`: True，把不同的参数类型得到的结果分开存储，如浮点型和整型（1.0和1）区分开来。
- `lru_cache`: 使用字典存储结果，根据调用时传入的定位参数和关键字参数创建。被装饰的函数，它的所有的参数都必须是可散列的。

In [54]:
# 使用缓存实现
import functools

@functools.lru_cache()   # 需要添加括号，functools.lru_cache()为需要传参的装饰器
@time_count
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

fibonacci(20)

<0.0000000000> fibonacci(0) -> 0
<0.0000000000> fibonacci(1) -> 1
<0.0010001659> fibonacci(2) -> 1
<0.0000000000> fibonacci(3) -> 2
<0.0010001659> fibonacci(4) -> 3
<0.0000000000> fibonacci(5) -> 5
<0.0010001659> fibonacci(6) -> 8
<0.0000000000> fibonacci(7) -> 13
<0.0010001659> fibonacci(8) -> 21
<0.0000000000> fibonacci(9) -> 34
<0.0010001659> fibonacci(10) -> 55
<0.0000000000> fibonacci(11) -> 89
<0.0010001659> fibonacci(12) -> 144
<0.0000000000> fibonacci(13) -> 233
<0.0010001659> fibonacci(14) -> 377
<0.0000000000> fibonacci(15) -> 610
<0.0010001659> fibonacci(16) -> 987
<0.0000000000> fibonacci(17) -> 1597
<0.0010001659> fibonacci(18) -> 2584
<0.0000000000> fibonacci(19) -> 4181
<0.0010001659> fibonacci(20) -> 6765


6765

### 单分派泛函数 `functools.singedpatch`

### 参数化装饰器